In [6]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

26 documents
6 classes ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
46 unique stemmed words ['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [8]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [9]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8420 - accuracy: 0.1154
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.8223 - accuracy: 0.1538
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7568 - accuracy: 0.2692
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7096 - accuracy: 0.2692
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7366 - accuracy: 0.3077
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6137 - accuracy: 0.3462
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6727 - accuracy: 0.3462
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5221 - accuracy: 0.5385
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4552 - accuracy: 0.5385
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4834 - accuracy: 0.6154
Epoch 11/200
6/6 [===========

6/6 [==============================] - 0s 2ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0521 - accuracy: 0.9615
Epoch 171/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0527 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0155 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0271 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0330 - accuracy: 0.9615
Epoch 175/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 176/200
6/6 [=====

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [15]:
p = bow("hello how are you", words)
print (p)
print (classes)

found in bag: hello
found in bag: how
found in bag: ar
found in bag: you
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1]
['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']


In [12]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[5.8687706e-06 3.0131866e-06 9.9993956e-01 2.9680974e-05 5.0006124e-06
  1.6814605e-05]]


In [22]:
# save model to file
# import pickle
# pickle.dump(model, open("katana-assistant-model.pkl", "wb"))

model.save("my_model.h5")

TypeError: can't pickle _thread.RLock objects

In [23]:
pickle_out=open("dict.pickle","wb")

In [24]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, pickle_out )

In [17]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [1]:
classify_local('how are you doing')


NameError: name 'classify_local' is not defined